### RAS XS Geopackage Viewer

An interactive viewer to show the versioned and cataloged HEC-RAS XS

Make sure to load the prerequisite credentials, then load the GLUE catalog.

In [ ]:
from pyiceberg.catalog import load_catalog

from icefabric.helpers import load_creds, load_pyiceberg_config, to_geopandas

# dir is where the .env file is located
load_creds()

# Loading the local pyiceberg config settings
pyiceberg_config = load_pyiceberg_config()

# Loading Glue Catalog
catalog = load_catalog("glue", **{"type": "glue", "glue.region": "us-east-1"})

Let's call the API to return the conflated cross-sections for a specific flowpath ID

In [ ]:
import geopandas as gpd
import httpx

from icefabric.schemas import XsType

# Set up parameters for the API call
flowpath_id = "20059822"
url = f"http://0.0.0.0:8000/v1/ras_xs/{flowpath_id}/"
schema_type = XsType.CONFLATED
params = {
    "schema_type": schema_type.value,
}
headers = {"Content-Type": "application/json"}

# Use HTTPX to stream the resulting geopackage response
with (
    httpx.stream(
        method="GET",
        url=url,
        params=params,
        headers=headers,
        timeout=60.0,  # GLUE API requests can be slow depending on the network speed. Adding a 60s timeout to ensure requests go through
    ) as response
):
    response.raise_for_status()  # Ensure the request was successful
    with open(f"ras_xs_{flowpath_id}.gpkg", "wb") as file:
        for chunk in response.iter_bytes():
            file.write(chunk)

# Load geopackage into geopandas
gdf = gpd.read_file(f"ras_xs_{flowpath_id}.gpkg")

Next, pull the reference divides and flowpaths for when we explore the data from the API

In [ ]:
from pyiceberg.expressions import In

# Pull and filter reference divides/flowpaths from the catalog
reference_divides = to_geopandas(
    catalog.load_table("conus_reference.reference_divides")
    .scan(row_filter=In("flowpath_id", gdf["flowpath_id"]))
    .to_pandas()
)
reference_flowpaths = to_geopandas(
    catalog.load_table("conus_reference.reference_flowpaths")
    .scan(row_filter=In("flowpath_id", gdf["flowpath_id"]))
    .to_pandas()
)

# Convert all data to the EPSG:4326 coordinate reference system
reference_divides = reference_divides.to_crs(epsg=4326)
reference_flowpaths = reference_flowpaths.to_crs(epsg=4326)
gdf = gdf.to_crs(epsg=4326)

Finally, project the conflated dataset over the references

In [ ]:
ref_div_ex = reference_divides.explore(color="grey")
ref_flo_ex = reference_flowpaths.explore(m=ref_div_ex, color="blue")

# View the data
gdf.explore(m=ref_flo_ex, color="black")

Now let's do the same thing, except filter the data so it lies inside a lat/lon bounding box - the following query gets all the data in the state of New Mexico

In [ ]:
from app.routers.ras_xs.router import BoundingBox

url = "http://0.0.0.0:8000/v1/ras_xs/within"
schema_type = XsType.REPRESENTATIVE
bbox = BoundingBox(min_lat=31.3323, min_lon=-109.0502, max_lat=37.0002, max_lon=-103.002)
params = {
    "schema_type": schema_type.value,
    "min_lat": bbox.min_lat,
    "min_lon": bbox.min_lon,
    "max_lat": bbox.max_lat,
    "max_lon": bbox.max_lon,
}

# Use HTTPX to stream the resulting geopackage response
with (
    httpx.stream(
        method="GET",
        url=url,
        params=params,
        headers=headers,
        timeout=60.0,  # GLUE API requests can be slow depending on the network speed. Adding a 60s timeout to ensure requests go through
    ) as response
):
    response.raise_for_status()  # Ensure the request was successful
    with open("ras_xs_bbox.gpkg", "wb") as file:
        for chunk in response.iter_bytes():
            file.write(chunk)

# Load geopackage into geopandas
gdf_bbox = gpd.read_file("ras_xs_bbox.gpkg")

In [ ]:
# Pull and filter reference divides/flowpaths from the catalog
reference_divides_bbox = to_geopandas(
    catalog.load_table("conus_reference.reference_divides")
    .scan(row_filter=In("flowpath_id", gdf_bbox["flowpath_id"]))
    .to_pandas()
)
reference_flowpaths_bbox = to_geopandas(
    catalog.load_table("conus_reference.reference_flowpaths")
    .scan(row_filter=In("flowpath_id", gdf_bbox["flowpath_id"]))
    .to_pandas()
)

# Convert all data to the EPSG:4326 coordinate reference system
reference_divides_bbox = reference_divides_bbox.to_crs(epsg=4326)
reference_flowpaths_bbox = reference_flowpaths_bbox.to_crs(epsg=4326)
gdf_bbox = gdf_bbox.to_crs(epsg=4326)

In [ ]:
ref_div_ex_bbox = reference_divides_bbox.explore(color="grey")
ref_flo_ex_bbox = reference_flowpaths_bbox.explore(m=ref_div_ex_bbox, color="blue")

# View the data
gdf_bbox.explore(m=ref_flo_ex_bbox, color="black")